Jupyter notebook is useful as we can keep datasets in memory between training runs

## Load datasets:

In [7]:
# Don't cache my other files https://stackoverflow.com/a/57245926
%reload_ext autoreload
%autoreload 2

from train import train_model
from torch.utils.data import random_split, ConcatDataset
from face_dataset import *
from label_funcs import *
import preprocessor
from ds_transforms import *

# Define dataset parameters
ds_size = 6667
equalise = False
processor = preprocessor.processor(w=84, h=84)
transform = lenet_transform(size=84)
classes = 100
equlalise = False
augment = False
print_errors = False

# adience_gender_ds = SlowDataset(
#     'C:\\Users\\jckpn\\Documents\\YEAR 3 PROJECT\\implementation\\source\\datasets\\training\\adience',
#     age_label_all, transform, processor, ds_size=50, equalise=True, classes=2, augment=False, print_errors=False)

imdb_gender_ds = FastDataset(
    'C:\\Users\\jckpn\\Documents\\YEAR 3 PROJECT\\implementation\\source\\other\\imdb_crop',
    age_label_all, transform, processor, ds_size, print_errors, equalise=equalise, augment=augment, classes=classes)

wiki_gender_ds = FastDataset(
    'C:\\Users\\jckpn\\Documents\\YEAR 3 PROJECT\\implementation\\source\\datasets\\training\\raw_imdbwiki_crop\\wiki_crop',
    age_label_all, transform, processor, ds_size, print_errors, equalise=equalise, augment=augment, classes=classes)

utkface_gender_ds = FastDataset(
    'C:\\Users\\jckpn\\Documents\\YEAR 3 PROJECT\\implementation\\source\\utkface_itw',
    age_label_all, transform, processor, ds_size, print_errors, equalise=equalise, augment=augment, classes=classes)
    
train_val_set = ConcatDataset([wiki_gender_ds, imdb_gender_ds, utkface_gender_ds])

# Split dataset into training and validation sets
val_split_ratio = 0.2
val_size = int(val_split_ratio * len(train_val_set))
train_size = len(train_val_set) - val_size
train_set, val_set = random_split(train_val_set, [train_size, val_size])
print(f'Split dataset into {len(train_set)} training and {len(val_set)} validation examples')

test_set = val_set

Reading and processing 6667 files from C:\Users\jckpn\Documents\YEAR 3 PROJECT\implementation\source\other\imdb_crop...



6667 items successfully prepared

Reading and processing 6667 files from C:\Users\jckpn\Documents\YEAR 3 PROJECT\implementation\source\datasets\training\raw_imdbwiki_crop\wiki_crop...



6667 items successfully prepared

Reading and processing 6667 files from C:\Users\jckpn\Documents\YEAR 3 PROJECT\implementation\source\utkface_itw...



6667 items successfully prepared

Split dataset into 16001 training and 4000 validation examples


## Train desired models:

In [9]:

from torch.nn import CrossEntropyLoss, MSELoss  # classif. -> CEL, regress. -> MSE
from torch.optim import Adam, SGD
from networks import *
import tests

model = train_model(
    model=LeNet(1), 
    train_set=train_set,
    val_set=val_set,
    loss_fn=MSELoss(),
    optim_fn=SGD,
    learning_rate=0.001,
    model_save_dir='../models')

tests.mae(model, test_set, print_results=True)


TRAINING MODEL LeNet-1_2604-1346.pt WITH PARAMS:
 - Architecture: LeNet
 - Learning rate: 0.001
 - Optimizer: SGD
 - Loss function: MSELoss()
 - Other notes: None

+---------------+---------------+---------------+---------------+---------------+
|         EPOCH | EXAMPLES SEEN |    TRAIN LOSS |      VAL LOSS |  ELAPSED TIME |
+---------------+---------------+---------------+---------------+---------------+


|             1 |         16001 |        19.926 |        12.810 |          0:41 |


| *           2 |         32002 |        12.980 |        13.594 |          1:22 |


| **          3 |         48003 |        13.040 |        13.413 |          2:07 |


Epoch 4:  16%|█▋        | 82/501 [00:06<00:31, 13.13it/s]